In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import dlib
import rospy

ImportError: No module named rospy

In [ ]:
video_capture = cv2.VideoCapture(0)
win = dlib.image_window()
detector = dlib.get_frontal_face_detector()

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "../model/model.ckpt")

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    
    try:
        win.set_image(frame)
    except:
        pass

In [ ]:
# initiliaze
rospy.init_node('GoForward', anonymous=False)

# tell user how to stop TurtleBot
rospy.loginfo("To stop TurtleBot CTRL + C")

# What function to call when you ctrl + c    
rospy.on_shutdown(self.shutdown)

# Create a publisher which can "talk" to TurtleBot and tell it to move
# Tip: You may need to change cmd_vel_mux/input/navi to /cmd_vel if you're not using TurtleBot2
self.cmd_vel = rospy.Publisher('cmd_vel_mux/input/navi', Twist, queue_size=10)

#TurtleBot will stop if we don't keep telling it to move.  How often should we tell it to move? 10 HZ
r = rospy.Rate(10);

# Twist is a datatype for velocity
move_cmd = Twist()
# let's go forward at 0.2 m/s
move_cmd.linear.x = 0.2
# let's turn at 0 radians/s
move_cmd.angular.z = 0


while not rospy.is_shutdown():
    self.cmd_vel.publish(move_cmd)
    r.sleep()
                        

In [ ]:
def shutdown(self):
    # stop turtlebot
    rospy.loginfo("Stop TurtleBot")
# a default Twist has linear.x of 0 and angular.z of 0.  So it'll stop TurtleBot
    self.cmd_vel.publish(Twist())
# sleep just makes sure TurtleBot receives the stop command prior to shutting down the script
    rospy.sleep(1)